In [50]:
import ee
import datetime
import os
from os import times
from IPython.display import Image
import folium

#ee.Authenticate()
ee.Initialize()

In [44]:
# Define the area of interest (AOI) as a polygon
aoi = ee.Geometry.Polygon([[-7.791558613934031,33.01490300783628], [-7.792003860630503,33.01324314924896], [-7.788286318935862,33.01286529089581], [-7.788710107960215,33.014507163639024], [-7.791558613934031,33.01490300783628]]) 

aoi2 = ee.Geometry.Polygon(
        [[[-7.804606875208133, 33.018938747086196],
          [-7.8049501979620395, 32.985827153066126],
          [-7.717746218469852, 32.98913887168102],
          [-7.731479128626102, 33.02268101491522]]])
# Define the time range
start_date = '2023-01-01'
end_date = '2023-01-30'

# Define the Sentinel-2 collection
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR')

In [45]:
# Filter the collection by the AOI and time range
filtered_collection = s2_collection.filterBounds(aoi2).filterDate(start_date, end_date)

# Get the number of images in the collection
num_images = filtered_collection.size().getInfo()
print('Number of images found:', num_images)

Number of images found: 6


In [47]:
# Define the band names for Sentinel-2
band_names = ['B4', 'B3', 'B2']

# Define a function to calculate NDVI from an image
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Iterate over the images in the collection and download them
for i in range(num_images):
    # Get the image
    image = ee.Image(filtered_collection.toList(num_images).get(i))

    # Get the image ID and date
    image_id = image.id().getInfo()
    image_date = datetime.datetime.utcfromtimestamp(image.get('system:time_start').getInfo()/1000.0).strftime('%Y-%m-%d')

    # Clip the image to the AOI and select the relevant bands
    image = image.clip(aoi2).select(band_names)
    
    # Calculate the NDVI and select the NDVI band
    #image = calculate_ndvi(image).select('NDVI')

    #image = image.select(['B4', 'B3', 'B2']).toFloat()
    #image = image.multiply(255).toUint8()

    # Define the export parameters
    export_params = {
        'image': image,
        'description': 'Sentinel2_' + image_id + '_' + image_date,
        'folder': 'Google Earth',
        'scale': 10, # Replace with your desired scale
    }

    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

    # Wait for the task to complete
    print('Exporting image', i+1, 'of', num_images, '- ID:', image_id, '- Date:', image_date)


Exporting image 1 of 6 - ID: 20230101T111451_20230101T111449_T29SPS - Date: 2023-01-01
Exporting image 2 of 6 - ID: 20230106T111349_20230106T111343_T29SPS - Date: 2023-01-06
Exporting image 3 of 6 - ID: 20230111T111431_20230111T112140_T29SPS - Date: 2023-01-11
Exporting image 4 of 6 - ID: 20230116T111319_20230116T111315_T29SPS - Date: 2023-01-16
Exporting image 5 of 6 - ID: 20230121T111351_20230121T112057_T29SPS - Date: 2023-01-21
Exporting image 6 of 6 - ID: 20230126T111239_20230126T111234_T29SPS - Date: 2023-01-26


In [49]:
image = ee.Image(filtered_collection.toList(num_images).get(1))

#image = image.clip(aoi)
#image = calculate_ndvi(image).select('NDVI')

image = image.clip(aoi2).select(band_names)
# Reduce the image resolution
#reduced_image = image.reduceResolution(ee.Reducer.mean(), maxPixels=65535)

# Get a thumbnail URL for the reduced image
thumbnail_url = image.getThumbUrl({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'dimensions': 500})

# Display the thumbnail
Image(url=thumbnail_url)

In [80]:
# Center the map on the polygon
Map = folium.Map(location=[40, -100], zoom_start=15, crs='EPSG4326')
#Map.center_object(aoi2, 14)
folium.Marker(location=[40, -100]).add_to(Map)

# Add the polygon layer to the map
#Map.add_child(folium.GeoJson(data=image.getInfo(), name='Polygon Layer'))
display(Map)